In [1]:
train_dataset_path = './DATA_CHAMBER_2021/train'
test_dataset_path = './DATA_CHAMBER_2021/test'

In [2]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [3]:
os.listdir(train_dataset_path)

['4C', '2C', '3C', '.DS_Store']

In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
train_datasets = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = transform)
test_datasets = torchvision.datasets.ImageFolder(root = test_dataset_path, transform = transform)

In [6]:
train_loader = DataLoader(train_datasets, batch_size = 8, shuffle = True, num_workers = 2)
test_loader = DataLoader(test_datasets, batch_size = 8, shuffle = False, num_workers = 2)

In [7]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)
set_device()

device(type='cpu')

In [8]:
def train_model(model, train_loader, criterion, optimizer, n_epochs):
    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch+1, n_epochs))
        print('-' * 10)
        
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
        
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total
        print('{} loss: {:.4f}, acc: {:.4f}'.format(total, epoch_loss, epoch_acc))

    return model

In [9]:
resnet50_model = models.resnet50(pretrained=True)
num_ftrs = resnet50_model.fc.in_features
number_of_classes = 3
resnet50_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = set_device()
resnet_50_model = resnet50_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet50_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [10]:
train_model(resnet50_model, train_loader, loss_fn, optimizer, 15)

Epoch 1/15
----------
6717 loss: 0.5238, acc: 82.2540
Epoch 2/15
----------
6717 loss: 0.1183, acc: 96.0846
Epoch 3/15
----------
6717 loss: 0.0726, acc: 97.7669
Epoch 4/15
----------
6717 loss: 0.0845, acc: 97.2905
Epoch 5/15
----------
6717 loss: 0.0777, acc: 97.5138
Epoch 6/15
----------
6717 loss: 0.0679, acc: 97.7817
Epoch 7/15
----------
6717 loss: 0.0457, acc: 98.6155
Epoch 8/15
----------
6717 loss: 0.0729, acc: 97.8711
Epoch 9/15
----------
6717 loss: 0.0552, acc: 98.3624
Epoch 10/15
----------
6717 loss: 0.0585, acc: 98.3773
Epoch 11/15
----------
6717 loss: 0.0561, acc: 98.4219
Epoch 12/15
----------
6717 loss: 0.0579, acc: 98.5112
Epoch 13/15
----------
6717 loss: 0.0670, acc: 98.2135
Epoch 14/15
----------
6717 loss: 0.0523, acc: 98.5410
Epoch 15/15
----------
6717 loss: 0.0823, acc: 97.6924


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
def test_model(model, test_loader, criterion, optimizer):
    labels_input=list()
    labels_output=list()
    vid_id = list()
        
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels_input= labels_input + labels.tolist()
    
        outputs = model(images)
            
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
            
        labels_output= labels_output + preds.tolist()
    return labels_input,labels_output

y_true,y_pred = test_model(resnet50_model, test_loader, loss_fn, optimizer)

In [12]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(classification_report(y_true,y_pred))
accuracy_score(y_true, y_pred)

              precision    recall  f1-score   support

           0       0.73      1.00      0.84       409
           1       1.00      0.70      0.82       367
           2       1.00      0.95      0.97       831

    accuracy                           0.90      1607
   macro avg       0.91      0.88      0.88      1607
weighted avg       0.93      0.90      0.91      1607



0.9041692594897324